# <B> Setup for SageMaker BYOM examples </B>
* Container: conda_python3

## AutoReload

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Set roles

In [31]:
from sagemaker import get_execution_role

In [32]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [33]:
from utils.iam import iam_handler

In [34]:
iam = iam_handler()

In [35]:
strID = "DJ"
strPrefix = f'{strID}-SM-IMD'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda

In [37]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    #"arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [38]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [39]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'd4a677da-4f58-4dd4-9fcc-98a1f1054c89', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd4a677da-4f58-4dd4-9fcc-98a1f1054c89', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:22 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': '127690d6-6c0e-4f01-a4c7-32d8e0fc6a98', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '127690d6-6c0e-4f01-a4c7-32d8e0fc6a98', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:22 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
{'ResponseMetadata': {'RequestId': '94b754f3-1f46-4aaf-8428-5c2f9a651e45', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '94b754f3-1f46-4aaf-8428-5c2f9a651e45', 'content-type': 

### 1.4 role-codebuild

In [40]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [41]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'e4ff3a8f-a2be-40fe-9bc8-6a104f9552a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e4ff3a8f-a2be-40fe-9bc8-6a104f9552a4', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:23 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSSMFullAccess
{'ResponseMetadata': {'RequestId': '919fa100-84aa-4e8d-8619-9285c309800a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '919fa100-84aa-4e8d-8619-9285c309800a', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:23 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': 'bdf1b27d-7310-4d1e-9d93-14d1e938d0d3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bdf1b27d-7310-4d1e-9d93-14d1e938d0d3', 'content-type': 'text/xml', 'content

### 1.5 role-codepipeline

In [42]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [43]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'f591bafa-bd05-4f95-900f-167438a1bccd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f591bafa-bd05-4f95-900f-167438a1bccd', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:24 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '1c75acb7-e22f-4713-9117-994d9be84c69', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1c75acb7-e22f-4713-9117-994d9be84c69', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:24 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeCommitFullAccess
{'ResponseMetadata': {'RequestId': '83b24a40-bbca-4646-96bc-0c920654d317', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '83b24a40-bbca-4646-96bc-0c920654d317', 'content-type': 'text/xml', '

### 1.6 role-eventbridge

In [44]:
listService = ["events.amazonaws.com"]
strRoleName = "-".join([strPrefix, "EventBridgeRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
]
strDescription="Role for EventBridge"

In [45]:
eventBridgeRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {eventBridgeRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-IMD-EventBridgeRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'cc07dbce-69f0-46ed-a71b-1be7167822f2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cc07dbce-69f0-46ed-a71b-1be7167822f2', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Thu, 04 May 2023 15:30:25 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess
  Service name: ['events.amazonaws.com'], 
  Role name: DJ-SM-IMD-EventBridgeRole, 
  Policys: ['arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole


## 2. Set default parameters

In [46]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [47]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [48]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [49]:
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
#print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
#print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))

REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-419974056037
SAGEMAKER-ROLE-ARN:  arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID:  419974056037
LAMBDA-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-LabmdaRole
CODE-EVENTBRIDGE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-IMD-EventBridgeRole
